In [3]:
import numpy as np
import jieba
import json
import codecs
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from gensim import models

In [4]:
model = models.Word2Vec.load('./model/finance.model')

In [2]:
df = pd.read_csv(r'G:\BaiduNetdiskDownload/financezhidao_filter.csv').fillna('')
df = df[df['is_best'] == 1]

In [3]:
df.head()

,title,question,reply,is_best
92,天*湖*小*的房子首付多少，能用公积金贷款,,由于各城市相关政策有所不同，具体您所在当地是否有开展公积金贷款业务，以及相关业务规定，您可以...,1
94,30万大额存单利率多少呢，我目前有30多万，想存,,,1
106,43###00日元多少人民币,,由于外汇汇率是实时变动的，您可以登录我行主页，点击右侧“实时金融信息”下“外汇实时汇率”查看...,1
108,定期没到期手机能不能转账,定期没到期手机能不能转账,【整存整取已到期转存】到期自动转存后的整存整取，可通过柜台、电话银行、网上银行、手机银行全额...,1
109,信用卡太多是不是不利于提额,,招行信用卡信用固定额度调整，目前可通过三种方式申请：1、您可以在持卡一段时间后，在我行客服热...,1


In [114]:
stop_words = ''.join(codecs.open('./input/stop_words.txt', encoding='utf-8').readlines()).split('\n')

In [6]:
title = df.title.tolist()

In [7]:
title = ','.join(title)

In [8]:
words = [t for t in jieba.cut(title) if t not in stop_words]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Zn\AppData\Local\Temp\jieba.cache
Loading model cost 0.546 seconds.
Prefix dict has been built succesfully.


In [9]:
word_dict = {}
for word in words:
    if word not in word_dict:
        word_dict[word] = 1
    else:
        word_dict[word] += 1

In [15]:
freq_word_df = pd.DataFrame.from_dict(word_dict,orient='index')

In [18]:
freq_word_df.to_csv('similar_words.csv', index=False)

In [25]:
freq_word_df.sort_values(0, inplace=True, ascending=False)

In [26]:
freq_word_df

,0
信用卡,89497
贷款,73255
银行,36116
银行卡,31037
招商银行,24827
钱,23601
利息,19355
办,19331
万,18238
办理,17652


In [3]:
def cluster_agt(sentence, model, n_clusters):
#     sentence = [s for s in sentence if is_q(s, rule_())]  
    vecs = np.array([get_vec(s, model) for s in sentence])
    clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward').fit(vecs)
    cluster_result = pd.DataFrame({'sentence': pd.Series(sentence), 'cluster_labels': pd.Series(clustering.labels_)})
    cluster_result = cluster_result.fillna('999')
    cluster_result['cluster_labels'] = cluster_result['cluster_labels'].astype('int32')
    return cluster_result

In [4]:
def get_vec(sentence, model):
    vec = np.zeros(model.vector_size)
    n = 0
    for word in jieba.cut(sentence):
        try:
            vec += model.wv[word]
            n += 1
        except KeyError:
            pass
    return vec if n==0 else vec/n 

In [7]:
q = df[df['title'].str.contains('信用卡')].title.tolist()

In [7]:
result = cluster_agt(q, model, 40)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Zn\AppData\Local\Temp\jieba.cache
Loading model cost 0.526 seconds.
Prefix dict has been built succesfully.


In [8]:
result = result[['sentence', 'cluster_labels']]

In [9]:
result.cluster_labels.value_counts()

0     1874
10    1425
21    1370
17    1308
1     1234
12     902
7      894
13     888
20     832
18     776
2      761
11     746
14     677
22     653
25     636
30     624
36     570
15     567
3      522
19     465
4      419
27     409
9      390
23     371
5      355
32     350
37     344
34     333
16     307
38     276
28     276
6      272
35     203
29     200
8      137
31     134
33      90
24      69
39      51
26      39
Name: cluster_labels, dtype: int64

In [8]:
# 26,22,2,12,17 删除
# 24 免息
# 31,6,14,1,21,33 申请、办理
# 8 挂失、补卡
# 29 出境
# 18 注销
# 20 推荐
# 35,27,22 提额、额度
# 28 被封
# 14 没通过
# 38 转账
# 0  还款
# 16 无工作怎么办理
# 30,10 逾期
# 34 取出来
# 25,36 多张信用卡
# 37,11 贷款
# 18 年费
# 32,9,3 其他
# 19,15,19,2,13,7,1 杂
# 11 贷款渠道
# 5  退款
# 23,39 信用卡好处
# 4  信用卡等级咨询
result[result['cluster_labels'] == 1].sample(15)

NameError: name 'result' is not defined

In [26]:
# df1.cluster_labels.replace([19,15,19,2,13,7,1], '杂', inplace=True)
# df1.cluster_labels.replace([23,39], '好处', inplace=True)
# df1.cluster_labels.replace([5], '退款', inplace=True)
# df1.cluster_labels.replace([4], '等级', inplace=True)
# df1.cluster_labels.replace([11,37], '贷款渠道', inplace=True)
# df1.cluster_labels.replace('多张卡', '其他', inplace=True)
# df1.cluster_labels.replace([34], '套现', inplace=True)
# df1.cluster_labels.replace([14], '没通过', inplace=True)
# df1.cluster_labels.replace([0], '还款', inplace=True)
# df1.cluster_labels.replace([16], '无工作', inplace=True)
# df1.cluster_labels.replace('没通过', '无工作', inplace=True)
# df1.cluster_labels.replace([38], '转账', inplace=True)
# df1.cluster_labels.replace([28], '被封', inplace=True)
# df1.cluster_labels.replace([20], '推荐', inplace=True)
# df1.cluster_labels.replace([18], '注销', inplace=True)
# df1.cluster_labels.replace([29], '出境', inplace=True)
# df1.cluster_labels.replace([30,10], '逾期', inplace=True)
# df1.cluster_labels.replace([8], '挂失', inplace=True)
# df1.cluster_labels.replace([31,6,14,1,21,33], '申请', inplace=True)
# df1.cluster_labels.replace([24], '免息', inplace=True)
# df1.cluster_labels.replace([35,27,22], '额度', inplace=True)
df1.cluster_labels.replace([25, 36], '多张卡', inplace=True)

D:\Tools\Anaconda\lib\site-packages\pandas\core\generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [23]:
df1 = df1[-df1['cluster_labels'].isin([26,22,2,12,17])]

In [5]:
df1 = pd.read_csv('credit_card_small_version.csv')

In [39]:
df1.to_csv('credit_card_small_version.csv', index=False)

In [6]:
df0 = pd.read_csv('credit_card.csv')

In [40]:
df1.cluster_labels.value_counts()

杂       4809
逾期      2049
还款      1874
申请      1866
额度      1265
其他      1262
多张卡     1206
贷款渠道    1090
无工作      984
推荐       832
注销       776
好处       422
等级       419
退款       355
套现       333
被封       276
转账       276
出境       200
挂失       137
免息        69
Name: cluster_labels, dtype: int64

In [45]:
# q = df[df['cluster_labels'].isin([3,28,21,9,38,14,5,8,19,18,24,2,38,1,13,35])].sentence.tolist()
df0.cluster_labels.replace([3,28,21,9,38], '申请', inplace=True)
df0.cluster_labels.replace([13], '额度', inplace=True)
df0.cluster_labels.replace([4,11], '逾期', inplace=True)
df0.cluster_labels.replace([34], '注销', inplace=True)
df0.cluster_labels.replace([22], '申请', inplace=True)
df0.cluster_labels.replace([30], '推荐', inplace=True)
df0.cluster_labels.replace([5,10], '申请', inplace=True)
df0.cluster_labels.replace([36], '年龄', inplace=True)
df0.cluster_labels.replace([25], '条件', inplace=True)
df0.cluster_labels.replace([32,14], '逾期', inplace=True)
df0.cluster_labels.replace([15,16], '进度', inplace=True)
df0.cluster_labels.replace([31], '分期', inplace=True)
df0.cluster_labels.replace([33], '利息', inplace=True)
df0.cluster_labels.replace([37,34], '额度', inplace=True)
df0.cluster_labels.replace([29,20], '最低还款', inplace=True)
df0.cluster_labels.replace([17], '积分', inplace=True)
df0.cluster_labels.replace([0], '手续费', inplace=True)
df0.cluster_labels.replace([12], '账单日', inplace=True)
df0.cluster_labels.replace([7], '邮箱密码', inplace=True)
df0.cluster_labels.replace([27,1], '转账', inplace=True)

In [47]:
df = pd.concat([df0, df1])

In [88]:
df.cluster_labels.value_counts()

杂       11946
申请      11623
逾期       7666
转账       3530
额度       2998
无工作      2547
还款       1874
推荐       1806
注销       1419
挂失       1326
最低还款     1292
其他       1262
多张卡      1206
贷款渠道     1090
进度       1031
邮箱密码      782
账单日       698
手续费       680
积分        483
利息        451
好处        422
等级        419
分期        414
退款        355
条件        336
套现        333
年龄        322
被封        276
23        212
出境        200
免息         69
Name: cluster_labels, dtype: int64

In [52]:
df[df['cluster_labels'] == '转账'].sample(10)

,sentence,cluster_labels
28748,信用卡刷卡免年费有金额限制吗？,转账
22259,招行信用卡网上支付算是刷卡消费吗,转账
12659,信用卡可以转账用么？,转账
22455,我没有添加信用卡怎么礼品卡购买成功了,转账
26763,上个月20号刷信用卡失败显示已冲正需要多久退回啊,转账
8508,信用卡能转账到储蓄卡吗–手机爱问,转账
591,招商银行信用卡能查出转账到账信息吗,转账
33557,招商银行信用卡刷境外机要收手续费吗,转账
27016,你好请问一下手机分期付款没有还能不能办信用卡,转账
8607,哪些支付或者理财平台支持信用卡？,转账


In [ ]:
## 3、28、21、9、38    如何办理+货
## 13  额度相关
## 4、11    逾期
## 34  挂失、注销、解封
## 22  身份证办理
## 30  哪个信用卡好
## 5、10    如何办理
## 14  欠、逾期
## 36  年龄
## 25  办理信用卡的条件和材料
## 32  逾期、不良
## 16、15  进度、审核
## 31  手续费、分期
## 33  利息、借多少钱
## 37  提升额度
## 29、20  最低还款+杂
## 17  积分、年费
## 39、26  哪里可以办理（删除）
## 34  有多少额度
## 0    手续费、取现金
## 12  账单日
##  7   邮箱+密码相关
##  27、1 信用卡消费、充值、转账

In [94]:
df.to_csv('clusted.csv', index=False)

In [12]:
df = pd.read_csv('drop_finance.csv')

In [13]:
df.shape

(36852, 2)

In [5]:
similar_json = json.load(codecs.open('./model/finance.json', 'r', encoding='UTF-8'))

In [20]:
jieba.lcut('银联要信用卡吗')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Zn\AppData\Local\Temp\jieba.cache
Loading model cost 0.529 seconds.
Prefix dict has been built succesfully.


['银联', '要', '信用卡', '吗']

In [28]:
jieba.lcut('通付可以刷自己的信用卡吗')

['通付', '可以', '刷', '自己', '的', '信用卡', '吗']

In [8]:
similar_json['信用卡']

{'lqzn': 0.4851926267147064,
 '不提': 0.4871380031108856,
 '你好': 0.4961557984352112,
 '储蓄卡': 0.5094289779663086,
 '即取': 0.5097764730453491,
 '即可': 0.4847288727760315,
 '第一次': 0.5016353130340576,
 '银行': 0.4976101815700531}

In [26]:
for sentence in df.sentence.tolist():
    score = simq('信用卡怎么办理', sentence)
    if score > 0.8:
        print(sentence, score)

D:\Tools\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\Tools\Anaconda\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


怎么办理信用卡 1.0
信用卡怎么申请办理 0.887159988284111
信用卡办理 1.0
办理信用卡 1.0
信用卡怎么申请 0.8495466510454813
申请办理信用卡 0.8495466510454813
信用卡信用卡 1.0
信用卡怎么办理 1.0
怎么申请办理信用卡 0.887159988284111
怎么申请信用卡 0.8495466510454813
第一次办理信用卡 0.8338784376780192
信用卡申请怎么申请 0.8495466510454813
信用卡怎么办理信用卡 1.0


In [31]:
df[df['sentence'] == '怎么办理信用卡']

,sentence,cluster_labels
1803,怎么办理信用卡,申请
